In [ ]:
# Import libraries

import os
import json
import pandas as pd
from elasticsearch import Elasticsearch, ElasticsearchWarning
from tqdm.auto import tqdm
import minsearch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read JSON document with generated ids

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
# Define the model to use

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
v = model.encode('I just discovered the course. Can I still join?')
len(v) # dimensions

384

In [6]:
# Initialize elastic search
es_client = Elasticsearch('http://localhost:9200') 

# Create an index mapping the question, text and question_text embeddings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
# Index the documents

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████████████| 948/948 [04:42<00:00,  3.36it/s]


In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████| 948/948 [03:19<00:00,  4.76it/s]


In [9]:
# Define the search query

def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [10]:
# Evaluate on questions only

def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

*Hit Rate (recall)*

In [13]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

*Mean Reciprocal Rank (mrr)*

In [14]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [15]:
# Load the ground truth CSV file into a DataFrame 

df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records') # list of dictionaries

In [15]:
ground_truth[-1]

{'question': 'How do you specify the variable file when running terraform destroy?',
 'course': 'mlops-zoomcamp',
 'document': '886d1617'}

In [16]:
# Function that evaluates the 

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [17]:
evaluate(ground_truth, question_vector_knn)

100%|████████████████| 4627/4627 [07:33<00:00, 10.20it/s]


{'hit_rate': 0.8232115841798141, 'mrr': 0.6735101251796741}

In [18]:
# Evaluate on the answer text only

def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)       

    return elastic_search_knn('text_vector', v_q, course)
    
evaluate(ground_truth, text_vector_knn)

100%|████████████████| 4627/4627 [07:02<00:00, 10.94it/s]


{'hit_rate': 0.8763777825805057, 'mrr': 0.7128502989698162}

In [19]:
# Evaluate on both the questions and answers

def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

100%|████████████████| 4627/4627 [06:48<00:00, 11.32it/s]


{'hit_rate': 0.9446725740220445, 'mrr': 0.8279830943976185}

The combination of question and text vectors performs the best

In [20]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            "script": { # calculates the combined cosine similarity score for the vectors and adds 1 to ensure the score is positive
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

evaluate(ground_truth, vector_combined_knn)

100%|████████████████| 4627/4627 [06:51<00:00, 11.24it/s]


{'hit_rate': 0.9023125135076724, 'mrr': 0.804480945176861}

This performs slightly worse than without the script